# Liquidity Mining Program

Global imports

In [4]:
#!pip install ciso8601

import requests
import ciso8601
from datetime import datetime, timedelta, timezone

url = 'https://vanir.s.singularity.so/v1/graphql'
headers = {
  'Content-Type' : 'application/json',
  'x-hasura-admin-secret' : '**********'
}


Snapshot calculation code

In [5]:
#get distinct addresses of sifchain liq mining users
def getDistinctAddresses():
  adr_query_str = '''
    query GetUniqueAddresses {
      events_betanet(where: {al_provider: {_is_null: false}}, distinct_on: [al_provider]) {
        al_provider
      }
    }
    '''
  adr_query_json = {"query": adr_query_str}
  adr_json = executeQuery(adr_query_json)
  unique_addresses = [i['al_provider'] for i in adr_json['events_betanet']]
  return unique_addresses

#get timestamp of when betanet started
def getFirstTimestamp():
  time_query_str = '''
      query GetFirstTimestamp {
        events_betanet(limit: 1, order_by: {timestamp: asc}) {
          timestamp
        }
      }
  '''
  time_query_json = {"query": time_query_str}
  time_json = executeQuery(time_query_json)
  first_timestamp = time_json['events_betanet'][0]['timestamp']
  ft = ciso8601.parse_datetime(first_timestamp)
  return ft

#current epoch time hardcoded to 10 minutes
def getEpochArray(epoch_time=10):
  ft = getFirstTimestamp()
  now = datetime.now(timezone.utc)
  
  epoch_arr=[]
  while (now > ft):
    new_now = now - timedelta(minutes=epoch_time)
    epoch_arr.append({'begin': new_now, 'end': now})
    now = new_now

  return epoch_arr

def getTokenPriceInRowan(token, timestamp):
  if type(timestamp) is str:
    timestamp = ciso8601.parse_datetime(timestamp)
  begin_ts = timestamp - timedelta(minutes=1)

  query_str = '''
      query GetTokenPrice {
        prices(where: {timestamp: {_gt: \"%s\"}}, order_by: {timestamp: asc}, limit: 2) {
          token_prices
          timestamp
        }
      }
  ''' %(begin_ts)

  query_json = {"query": query_str}
  price_json = executeQuery(query_json)
  price_json = price_json['prices'][0]['token_prices']
  token_price = price_json[token+'_rowan']
  return token_price

def computeLiquidityMiningData():
  adr_query_str_al = '''
    query GetUniqueAddresses_al {
      events_betanet(where: {al_provider: {_is_null: false}}, order_by: {timestamp: asc}) {
        al_provider
        al_token
        al_amount
        timestamp
      }
    }
    '''
  adr_query_json_al = {"query": adr_query_str_al}
  adr_json_al = executeQuery(adr_query_json_al)

  adr_query_str_rl = '''
    query GetUniqueAddresses_rl {
      events_betanet( where: {rl_provider: {_is_null: false}}, order_by: {timestamp: asc}) {
        rl_provider
        rl_token
        rl_amount
        timestamp
      }
    }
    '''

  adr_query_json_rl = {"query": adr_query_str_rl}
  adr_json_rl = executeQuery(adr_query_json_rl)
  return adr_json_al['events_betanet'], adr_json_rl['events_betanet']

def getUserSnapshot():
  #create snapshot scaffold
  distinct_addresses = getDistinctAddresses()
  epoch_array = getEpochArray()
  first_timestamp = getFirstTimestamp()

  snapshots={}
  for address in distinct_addresses:
    snapshots[address] = [0] * (len(epoch_array)+1)
  
  #al = added liquidity, rl = removed liquidity
  al_data, rl_data = computeLiquidityMiningData()

  #populate snapshot scaffold

  #iterate over liquidity add events
  for item in al_data:
    token = item['al_token']
    timestamp = ciso8601.parse_datetime(item['timestamp'])
    print(token, timestamp)
    address = item['al_provider']
    tokenAmount = item['al_amount']
    tokenPrice = getTokenPriceInRowan(token, timestamp)
    index = int((timestamp - first_timestamp).total_seconds() / (60*10))
    
    snapshots[address][index] += tokenAmount * tokenPrice
    #print(index, len(epoch_array), address, token, snapshots[address][index], snapshots[address])
  
  #iterate over remove liquidity events
  for item in rl_data:
    token = item['rl_token']
    timestamp = ciso8601.parse_datetime(item['timestamp'])
    print(token, timestamp)
    address = item['rl_provider']
    tokenAmount = item['rl_amount']
    tokenPrice = getTokenPriceInRowan(token, timestamp)
    index = int((timestamp - first_timestamp).total_seconds() / (60*10))
    try:
      snapshots[address][index] -= tokenAmount * tokenPrice
    except KeyError:
      snapshots[address] = [0] * (len(epoch_array)+1)
      snapshots[address][index] -= tokenAmount * tokenPrice
    #print(index, len(epoch_array), address, token, snapshots[address][index], snapshots[address])

  return snapshots

getUserSnapshot()
#getTokenPriceInRowan('c1inch', '2021-03-11T13:52:55.08538+00:00')
#getTokenPriceInRowan('ceth', '2021-02-14T17:17:55.661623+00:00')

NameError: name 'executeQuery' is not defined

Utilities

In [ ]:
def executeQuery(query_json):
  r = requests.post(url=url, json=query_json, headers=headers).json()
  return r['data']

def getDecimalDict():
  token_list_url = 'https://raw.githubusercontent.com/Sifchain/sifnode/develop/ui/core/src/assets.sifchain.mainnet.json'
  token_list = requests.get(token_list_url).json()
  token_decimals_dictionary = {'ceth':18}
  for token in token_list['assets']:
    try:
        token_decimals_dictionary[token['symbol'].lower()] = token['decimals']
    except KeyError as e:
        pass
  return token_decimals_dictionary

def getPools():
  url = 'https://api.sifchain.finance/clp/getPools'
  r = requests.get(url).json()
  pools = r['result']['Pools']
  token_decimals_dictionary = getDecimalDict()
  pool_data ={}
  for i in pools:
    d=i
    symbol = i['external_asset']['symbol']
    d['decimals'] = token_decimals_dictionary[symbol]
    pool_data[symbol] = d
  return pool_data

def calculatePoolShare(address):
  al_query_str = '''
  query GetLiquidityProvidersWithAmounts { events_betanet(
    where: {al_provider: {_eq: \"%s\"}}) 
    { al_provider al_amount al_token timestamp } }
    ''' % address

  al_query_json = {"query": al_query_str}
  al_json = executeQuery(al_query_json)

  rl_query_str = '''
  query GetLiquidityProvidersWithAmounts { events_betanet(
    where: {rl_provider: {_eq: \"%s\"}}) 
    { rl_provider rl_amount rl_token timestamp } }
    ''' % address

  rl_query_json = {"query": rl_query_str}
  rl_json = executeQuery(rl_query_json)

  pool_share = {}

  for i in al_json['events_betanet']:
    pool_share[i['al_token']] = 0

  for i in rl_json['events_betanet']:
    pool_share[i['rl_token']] = 0
  
  for i in al_json['events_betanet']:
    pool_share[i['al_token']] += i['al_amount']

  for i in rl_json['events_betanet']:
    pool_share[i['rl_token']] -= i['rl_amount']

  pools = getPools()
  pool_data=[]
  for token,amount in pool_share.items():
    d={}
    d['symbol'] = token
    d['amount'] = amount
    d['share'] = 0
    pool_data.append(d)
  
  for i in pool_data:
    nab = pools[i['symbol']]['native_asset_balance']
    decimals = pools[i['symbol']]['decimals']
    normalised_nab = float(nab)/10**(decimals)
    share = i['amount']/normalised_nab
    i['share'] = share
    
  return pool_data

def viewPoolShare():
  da = getDistinctAddresses()
  for i in da:
    print(i, calculatePoolShare(i))

viewPoolShare()
#calculatePoolShare('sif1sk4jsedru5ezt47hy52z3tkpts9m04henfrema')


Get all validators at current block height




In [ ]:
getValidatorQuery = {"query": "query GetValidators { validators_betanet(limit: 1 order_by: {height: desc}) { validators } }"}

r = requests.post(url=url, json=getValidatorQuery, headers=headers).json()
print(r['data']['validators_betanet'][0]['validators'])

Get list of all Liquidity Providers

In [ ]:
getLiquidityProviderQuery = {"query": "query GetLiquidityProviders { events_betanet(where: {type: {_eq: \"added_liquidity\"} timestamp: {}}) { al_provider } }"}

r = requests.post(url=url, json=getLiquidityProviderQuery, headers=headers).json()
al_providers = r['data']['events_betanet']
lp_arr = [i['al_provider'] for i in al_providers]
print(lp_arr)

Get Liquidity Providers with amounts

In [ ]:
getLiquidityProviderWithAmountsQuery= {"query": "query GetLiquidityProvidersWithAmounts { events_betanet(where: {type: {_eq: \"added_liquidity\"} timestamp: {}}) { al_provider al_amount al_token } }"}
r = requests.post(url=url, json=getLiquidityProviderQuery, headers=headers).json()
al_providers = r['data']['events_betanet']

print(r)

Calculate rewards

In [ ]:
#vesting multiplier (VM) Liquidity Lock Multiplier (LLM)
def calculateRewards(address, vm=1, llm=1):
  rowans = 833333
  pool_share = calculatePoolShare(address)
  rewards = []
  for i in pool_share:
    d=i
    d['rewards'] = rowans * i['share'] * vm * llm
    rewards.append(d)

  return rewards

calculateRewards('sif1sk4jsedru5ezt47hy52z3tkpts9m04henfrema')